### ✍️ Required libraries & settings

In [ ]:
import sys
sys.path.insert(1, '/home/sbanik@quansight.com/demo-dashboards')

import warnings
warnings.filterwarnings('ignore')

from intake_utils import (
    catalog_init,
    list_catalog,
    view_catalog)

import numpy as np
import panel as pn
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
from holoviews import opts,dim
from bokeh.models import HoverTool
hv.extension('bokeh')

### 👓 About dataset

- **Title :** Austin trees data
- **Brief description :** This dataset shows point locations of public trees inventoried by the City of Austin as of March 13th, 2020. Data is compiled from various sources: Development Services Department's Tree Division, AISD, Parks and Recreation Department, and Public Works Department's downtown tree inventory (2013). This is not a complete comprehensive inventory of all trees. Some errors and/or duplicate data may exist. For more information on Austin's urban forest, visit the U.S. Forest Service's Urban Forest Inventory and Analysis report: https://www.fs.usda.gov/treesearch/pubs/50393


#### 🗒 Columns in this Dataset:
- **GEOMETRY** : Feature's geometry type.
- **SPECIES** : Common species name of the tree.
- **DIAMETER** : Diameter of tree measure at breast height 4.5-feet from the ground. Units are measured in inches.
- **LATITUDE** : Estimated latitude of tree location.
- **LONGTITUDE** : Estimated longitude of tree location.
- **New Georeferenced Column** : Similar to GEOMETRY

### 📖 Reading via from `intake` catalog


In [ ]:
catalog = catalog_init()

In [ ]:
data_trees = catalog.austin_trees.read()
data_trees

### Data stats and additional info

In [ ]:
data_trees.info()

In [ ]:
data_trees.describe()

## Insights 🛠

### Various tree species and its abundance

In [ ]:
specie_abundance = data_trees.SPECIES.value_counts().reset_index().rename(columns={'index': 'Specie_name', 'SPECIES': 'Count'})

In [ ]:
data_trees.groupby(['SPECIES']).DIAMETER.mean().reset_index()

In [ ]:
specie_count_plot = hv.Table(
    specie_abundance, hv.Dimension('Specie_name'), 'Count')
specie_count_plot + hv.Curve(
    (specie_abundance.Specie_name, specie_abundance.Count),
    "Specie name", 
    "Count",
    label="Stats-WIP")

### Insights:
- `Southern Live Oak` is the most abundant specie in Austin

### 🧭 Insights (geo plot)

In [ ]:
import plotly.express as px
import pandas as pd

fig = px.scatter_geo(data_trees,lat='LATITUDE',lon='LONGTITUDE', hover_name="SPECIES")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()